In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians

In [2]:
@load "chimeraIPM_nopetsc.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any, Any} with 44 entries:
  "lamg_build"    => [1.07675, 1.11742, 1.13655, 1.13066, 1.09797, 1.09307, 1.1…
  "hypre_solve"   => [0.48, 0.52, 0.45, 0.54, 0.55, 0.49, 0.46, 0.45, 0.53, 0.5…
  "icc2_tot"      => [0.870099, 0.867364, 0.857776, 0.828449, 0.931116, 0.78020…
  "lamg_its"      => [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0  …  Inf,…
  "lamg_solve"    => [0.822568, 0.902733, 0.886765, 0.866826, 0.864624, 0.84768…
  "hypre_its"     => [8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0  …  9.0,…
  "cmg_solve"     => [0.584477, 0.592069, 0.600114, 0.573069, 0.584787, 0.57749…
  "ac_tot"        => [1.32341, 1.24779, 1.22765, 1.22806, 1.2321, 1.24706, 1.22…
  "cmg_build"     => [0.637418, 0.565644, 0.600613, 0.631511, 0.619733, 0.53641…
  "icc2_solve"    => [0.720296, 0.718707, 0.695286, 0.682965, 0.76934, 0.635247…
  "hypre_tot"     => [2.45, 2.47, 2.39, 2.51, 2.55, 2.5, 2.43, 2.43, 2.52, 2.54…
  "ac-s2m2_its"   => [14.0, 13.0, 14.0, 13.0, 13.0, 13.0, 14.0, 14.0, 13.0, 1

In [3]:
import Laplacians.initDictCol!
dics = [Dict(),Dict(),Dict(),Dict(),Dict()]

for key in keys(dic1)
    for i in 1:length(dics)
        initDictCol!(dics[i], key, typeof(dic1[key][1]))
        #@show dics[i]
    end
end

for i in 1:length(dic1["testName"])
    tn = dic1["testName"][i]
    curTargetEps = 0.0
    if occursin("e", (split(tn, ".")[4][4:end]))
        curTargetEps = parse(Float64, (split(tn, ".")[4][4:end]))
    else
        curTargetEps = parse(Float64, string(split(tn, ".")[4][4:end], ".", split(tn, ".")[5]))
    end
    curTargetEpsInd = round(Int, -log10(curTargetEps))
    for key in keys(dic1)
        if key == "names"
            continue
        end
        push!(dics[curTargetEpsInd][key], dic1[key][i])
    end
end

for i in 1:length(dics)
    dics[i]["names"] = dic1["names"]
end

In [4]:
function nanErr!(d)
    for (key, value) in d
        if occursin("err", key)
            for i in 1:length(value)
                if isnan(value[i])
                    d[key][i] = Inf
                end
            end
        end
    end
end

nanErr! (generic function with 1 method)

In [5]:
nanErr!.(dics);

In [6]:
using Printf

function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=3)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))
ffK(s) = (@sprintf "%.4g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)
ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe3(s) = (@sprintf "%.3g" sd(3,s)*1e3)

ffe3 (generic function with 1 method)

In [7]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = median(d[string(solver, "_tot")] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = quantile(d[string(solver, "_tot")] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver)
    if haskey(d, string(solver, "_tot")) && length(d[string(solver, "_tot")]) == length(d["ne"])
        tvn = maximum(d[string(solver, "_tot")] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

printMax (generic function with 1 method)

In [8]:
function printSolverCell(d, solver) 
    return string(printMedian(d, solver), "&", print75(d, solver), "&", printMax(d, solver))
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver) for solver in solvers]
    println(string(ffe3(1 / 10^i), "&", length(d["testName"]), "&", join(allCells, "&"), "\\\\"))
end

function printAllLines(dics, solvers)
    for i in 1:length(dics)
        printOneLine(dics[i], solvers, i)
    end
end

printAllLines (generic function with 1 method)

In [9]:
solvers=["ac", "ac-s2m2", "cmg2"]
printAllLines(dics, solvers)

100&28&0.68&0.774&1.2&1.27&1.79&2.39&1.6&1.83&Inf\\
10&27&0.678&0.777&1.15&1.27&1.8&2.36&1.62&1.97&2.55\\
1&27&0.68&0.763&1.14&1.25&1.79&2.38&1.56&1.91&2.51\\
0.1&27&0.683&0.784&1.15&1.27&1.79&2.36&1.55&1.89&2.71\\
0.01&19&0.681&0.777&1.15&1.28&1.78&2.37&1.57&1.87&2.33\\


In [10]:
solvers = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dics, solvers)

100&28&1.54&2.27&5.04&N/A&N/A&N/A&1.23&1.8&4.79\\
10&27&1.52&2.29&4.82&N/A&N/A&N/A&1.32&1.87&5.24\tnote{*}\\
1&27&1.52&2.32&4.94\tnote{*}&N/A&N/A&N/A&1.07\tnote{*}&1.83\tnote{*}&3.06\tnote{*}\\
0.1&27&1.53&2.29&4.86\tnote{*}&N/A&N/A&N/A&1.02&1.72&3.44\\
0.01&19&1.52&2.3&4.58\tnote{*}&N/A&N/A&N/A&0.962&1.73&3.26\\
